<img src="programmable.png" style="height:70%;width:70%;">

We start by importing our pip package, `modelzoo`. If you would like to follow along, feel free to do so using the Jupyter Notebooks provided. Make sure to run `pip install modelzoo` to install the package though!

In [ ]:
from modelzoo.admin import ModelZooConnection

## Connecting to a ModelZoo Instance
We first need to connect our client API to a modelzoo instance. It is important to note that we connect to the address of the gRPC server rather than the frontend's address.

In [ ]:
conn = ModelZooConnection(address="52.40.213.134:9000")

Let's take a look at all of the models!

In [ ]:
conn.list_all_models(expand_metadata=True)

We see all of the models from when we were on the UI, and all of the associated metadata!

Let's run a similar request to the one we did last time.

We'll be running Vision Classification on this image:
<img src="hd.jpg" style="width:50%;height:50%;">

In [ ]:
import modelzoo.sugar as sugar
payload = conn.image_inference("ImageNet Classification ResNet152", "hd.jpg")
sugar.table_input(payload.table)

Now, we will filter all of our models on some predicate, in this example, whether `ImageNet` is in the name, or its category is `vision classification`. This helps us narrow down what models on ModelZoo.Live actually pertain to the task we are trying to solve.

In [ ]:
models = conn.list_all_models(expand_metadata=True)
models_to_query = set()
g = []
for idx, df in models.groupby(level=0):
    if 'ImageNet' in idx:
        models_to_query.add(idx)
    df = df.droplevel(0)
    if df.loc['category', 'value'] == 'vision classification':
        models_to_query.add(idx)
models = list(models_to_query)
models

We'll run our models on some simple data at first - fairly clean images of food.

First, lets take a look at some of the images.

In [ ]:
import PIL
from IPython.display import display, Image

images = ['Cheeseburger.jpg', 'guacamole.jpg', 'butternut.jpeg']
def display_img(im):
    im = PIL.Image.open(im)
    bio = BytesIO()
    im.save(bio, format='png')
    display(Image(bio.getvalue(), format='png', width='300'))

for im in images:
    display_img(im)

In [ ]:
data = ['Cheeseburger.jpg', 'guacamole.jpg', 'gastronomy.jpg']
labels=['cheeseburger', 'guacamole', 'carpaccio']

In [ ]:
from modelzoo.sugar import table_input
import numpy as np
import itertools
from tqdm import tqdm
probs = {m:0 for m in models}
for model, (image, label) in tqdm(list(itertools.product(models, zip(data, labels)))):
        result = conn.image_inference(model, image)
        out = table_input(result.table)
        out.category = [word.split(',')[0] for word in out.category]
        out.probability = out.probability.astype(np.float32)
        if not out[out.category == label].empty:
            probs[model] += out[out.category == label]['probability'].values[0]
probs

We see that all of our models have a 100% accuracy on our small dataset.

Let's say we decide to ensemble all three models, and want to upload this back to ModelZoo.Live

# Uniform Interface for Model Evaluation
Lets walk through how to define a function to upload to ModelZoo.

In [ ]:
# from modelzoo.sugar import register_type, image_input, table_output
# @register_type(image_input, table_output)
from PIL import Image
def ensemble(inp: Image, metadata):
    from modelzoo.admin import ModelZooConnection
    from modelzoo.sugar import table_input
    import numpy as np
    import pandas as pd
    models = ['ImageNet Classification ResNet18', 'ImageNet Classification ResNet50',
              'ImageNet Classification ResNet152']
    tables = []
    for m in models:
        payload = conn.image_inference(m, inp)
        tables.append(sugar.table_input(payload.table))
    new_df = pd.concat(tables,sort=True).reset_index().drop('index', axis=1)
    new_df.probability = new_df.probability.astype(np.float32)
    new_df.probability /= np.sum(new_df.probability.values)
    new_df = new_df.drop(columns='rank')
    return new_df

In [ ]:
a = Image.open("gastronomy.jpg")
a

In [ ]:
ensemble(a, {"category": "vision classification ensemble"})